# Using a GAN to generate new MNIST data

This tutorial shows how a PT Series can be used to provide the latent space at the input of a generative adversarial network (GAN). A GAN consists of two models that are trained simultaneously: a generator that learns to generate images that are as similar as possible to those in the training dataset and a critic that, when given these generated images and real images, has to determine which images are real or fake. See [1] for the original GAN paper, and [2] and [3] for information about the specific type of GAN implemented in this repository.

<center><img src="figures/gan_drawing.png" alt="Hybrid GAN" width="700"/></center>
<center>Figure 1: Overview of the hybrid quantum/classical GAN.</center>

Whereas classical GANs use classical latent spaces at the input of the generator, for some datasets using a quantum device such as the PT Series allows us to generate higher quality data.

[1] Goodfellow, Ian J., et al. "Generative Adversarial Networks." Proceedings of the International Conference on Neural Information Processing Systems (NIPS 2014). pp. 2672–2680.

[2] Arjovsky, Martin, Soumith Chintala, and Léon Bottou. "Wasserstein generative adversarial networks." International conference on machine learning. PMLR, 2017.

[3] Gulrajani, Ishaan, et al. "Improved training of Wasserstein GANs." Advances in neural information processing systems 30 (2017).

In [ ]:
# First, perform the relevant imports and navigate to the root folder
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("..")

from ptseries.algorithms.gans import WGANGP
from ptseries.models import PTGenerator
from ptseries.common.logger import Logger
from ptseries.common.set_seed import set_seed
from tutorial_notebooks.utils.mnist_utils import load_mnist, plot_batch
from tutorial_notebooks.utils.mnist_models import Generator, Critic

set_seed(10)

### Load the data and prepare training logs

In this notebook, we use the MNIST dataset of handwritten digits. The following cell downloads this dataset to the SDK folder if it has not already been downloaded, prepares it as a PyTorch dataloader, and plots some images from the dataset.

In [ ]:
train_loader = load_mnist(batch_size=32)

# Plot images from the dataset
train_examples = next(iter(train_loader))
plot_batch(train_examples[:25])

We next create a logger, which will automatically keep track of training information and save it to a log folder in the SDK folder

In [ ]:
logger = Logger()
print("log folder : ", logger.log_folder)

### Create the quantum latent space and the GAN

A GAN consists of three elements: a latent space, a generator and a critic. In the code below, we use a PTGenerator as the latent space, and we import two ready-made generator and critic models from the `mnist_utils` folder.

Here, our PTGenerator uses a PT Series with 4 photons in 8 modes and randomly chosen beam splitter angles.

In [ ]:
# 1. Define the quantum latent space
input_state = [1, 0, 1, 0, 1, 0, 1, 0]
tbi_params = {"n_loops": 1}
latent_space = PTGenerator(input_state=input_state, tbi_params=tbi_params)

# 2. Define the generator
generator = Generator(len(input_state))

# 3. Define the critic
critic = Critic()

In [ ]:
print(latent_space)
print(generator)
print(critic)

### Train the algorithm


We now instantiate the GAN that we will use to train the models defined in the previous section. Note that a larger network, training for more iterations, and using a larger latent space will improve the results.

In [ ]:
gan = WGANGP(latent_space, generator, critic)

# 4. Train the gan
gan.train(
    train_loader,
    logger=logger,
    n_iter=1000,
    print_frequency=100,
    save_frequency=200,
)

### Generate new images

We now test the performance of the trained GAN by having it generate new images.

In [ ]:
# Generate and plot 25 images
imgs_fake = gan.generate_data(25)

plot_batch(imgs_fake)

The outputs of the trained GAN are starting to look like images, but they don't look like the images in the original training set yet.

# Improving the results

The images produced in the previous section are a bit blurry. This is normal! For the sake of having a simple notebook that runs quickly we used a small PT Series device and trained for relatively few iterations. Here, we go through some steps that will allow us to train a more powerful model.

### Using a larger latent space

First, we can use a more complex latent space with more photons and modes, which for real hardware can be made available with a PT-1 device upgrade. Alternatively, we can use a *tiling* method, where instead of sampling only once from the output of a PT-1 we sample multiple times and concatenate the results to form a single latent space vector. A new model that uses a latent space with 2 experimental results concatenated with each other is implemented in the following cell.

In [ ]:
# 1. Define the quantum latent space
input_state = [1, 0, 1, 0, 1, 0, 1, 0]
n_tiling = 2
latent_space = PTGenerator(input_state=input_state, n_tiling=n_tiling)

# 2. Define the generator
generator = Generator(len(input_state) * n_tiling)

# 3. Define the critic
critic = Critic()

### Training the latent space

Whereas previously the latent space was a fixed probability distribution produced by the PTGenerator, now we also jointly train the latent space with the generator and critic. An ORCA proprietary training method is implemented in the SDK, with details of this method to be discussed in an upcoming publication.

In addition to using a larger and trainable latent space, we now also train this new model with 1000 more iterations than we used previously. 

In [ ]:
gan = WGANGP(latent_space, generator, critic)

# 4. Train the gan
gan.train(
    train_loader,
    logger=logger,
    n_iter=2000,
    print_frequency=100,
    latent_update_frequency=1,  # The latent space is trained at every step
)

We can now inspect the results of this new trained model. We observe a significant improvement over our previous model! Note that longer training will lead to even better results.

In [ ]:
# Generate and plot 25 images
imgs_fake = gan.generate_data(25)

plot_batch(imgs_fake)